In [40]:
import pandas as pd
import os
import numpy as np
ind = 0
basins = [] 
d = {}
l = os.listdir('data')
for i in l:
    f = os.listdir('data/'+i)
    for j in f:
        j = int(j.split('_')[0])
        d[j] = i
        basins.append(j)
basins = sorted(basins)
pvals = [None]*671
df = pd.read_csv('non_temporal.csv')
df = df.drop('low_prec_timing',axis=1)
p_val = list(df['gauge_id'])

for i in basins:
    try:
        b = str(i)
        if(len(b)==7):
            b = '0'+b
        data = pd.read_csv('data/'+d[i]+'/'+b+'_lump_nldas_forcing_leap.txt',sep=r'\s+',skiprows=3)
        data = data.drop('Year',axis=1)
        data = data.drop('Mnth',axis=1)
        data = data.drop('Day',axis=1)
        data = data.drop('Hr',axis=1)
        prcp = data['PRCP(mm/day)']
        ind = p_val.index(i)
        pvals[ind] = list(prcp)
    except:
        continue
df['prcp'] = pvals

print(df)


     gauge_id  gauge_lat  gauge_lon  elev_mean  slope_mean  area_gages2  \
0     1013500   47.23739  -68.58264     250.31    21.64152      2252.70   
1     1022500   44.60797  -67.93524      92.68    17.79072       573.60   
2     1030500   45.50097  -68.30596     143.80    12.79195      3676.17   
3     1031500   45.17501  -69.31470     247.80    29.56035       769.05   
4     1047000   44.86920  -69.95510     310.38    49.92122       909.10   
..        ...        ...        ...        ...         ...          ...   
666  14309500   42.80400 -123.61091     709.83   110.42527       224.92   
667  14316700   43.34984 -122.72894     952.26   119.08920       587.90   
668  14325000   42.89150 -124.07065     656.53   124.96889       443.07   
669  14362250   42.15401 -123.07532     875.67   109.93127        41.42   
670  14400000   42.12344 -124.18731     625.31    98.81802       702.63   

     pet_mean  p_seasonality   aridity  soil_depth_pelletier  soil_porosity  \
0    1.971555       

In [41]:
df.to_csv('non_temp_prcp.csv',index=False)

In [53]:
vecs = df.values
vec1 = [vecs[0][1:-1]]
prcp1 = [list(vecs[0][-1])]
print(vec1)

[array([47.23739, -68.58264, 250.31, 21.64152, 2252.7, 1.97155451060917,
       0.187940258706929, 0.63055865946247, 7.40476190476191,
       0.461148751156712, 55.1569400320784, 5.37669779045723, 0.9063,
       3.34073185949204, 0.37164816263051, 1.84532416164806, -14.7019],
      dtype=object)]


In [58]:
from sklearn.metrics.pairwise import cosine_similarity

sims = {}
for i in vecs[1:]:
    try:
        vec_curr = [i[1:-1]]
        _id = i[0]
        prcp_curr = [list(i[-1])]
        cos_sim1 = cosine_similarity(vec1,vec_curr)
        cos_sim2 = cosine_similarity(prcp1,prcp_curr)
        cos_sim = (cos_sim1[0][0]+cos_sim2[0][0])/2
        if(float(cos_sim) in sims):
            sims[float(cos_sim)].append(_id)
        else:
            sims[float(cos_sim)] = [_id]
    except:
        continue
sim_scores = sorted(sims,reverse=True)[:10]
 

In [60]:
basins_sim = []
for i in sim_scores:
    for j in sims[i]:
        j = str(j)
        if(len(j)==7):
            j = '0'+j
        basins_sim.append(j)
print(basins_sim)

['01030500', '01031500', '01047000', '01022500', '01144000', '01057000', '04296000', '01052500', '01423000', '01548500']


In [61]:
sim_scores

[0.9072953403076828,
 0.86933246454252,
 0.8475450023729176,
 0.8431138486148112,
 0.77563303116785,
 0.6769450185183701,
 0.657348146494506,
 0.6564671365124594,
 0.6493108242803526,
 0.6431628837773671]